<a href="https://colab.research.google.com/github/luislizcano/seagrass-mapping-tb/blob/main/Preprocessing/Deglint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deglinting

This script apply a simple correction of sun glint on water surface of Sentinel-2 images. The output is exported to the EE Assets as an image.


Script by: Luis Lizcano-Sandoval (2021)<br/>
College of Marine Sciences, University of South Florida<br/>
luislizcanos@usf.edu / lsandoval@sig-gis.com<br/>
Updated: 12/26/2024

**Cite as:** *Lizcano-Sandoval, L.; Beck, M.W.; Scolaro, S.; Sherwood, E.T.; Muller-Karger, F. (2025) Cloud-Based Satellite Remote Sensing for Enhancing Seagrass Monitoring and Ecosystem Management (In preparation)*

### **Import packages**

In [2]:
## Authenticate your EE account
import ee
ee.Authenticate()
ee.Initialize(project='INSERT-EE-PROJECT-ID')

In [ ]:
## Clone github repo:
!git clone https://github.com/luislizcano/seagrass-mapping-tb.git

In [4]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.insert(0,'/content/seagrass-mapping-tb')
sys.path.append('/content/seagrass-mapping-tb/Preprocessing/')

Mounted at /content/drive


In [5]:
## Verify you loaded the EE module correctly:
import geemap
import functions
print('EE version: ',ee.__version__)

EE version:  1.4.3


### **Load image**


In [ ]:
## Define asset route to export image (to a folder or imageCollection)
asset_id = 'INSERT-EE-ASSETID/'

## Insert image id:
image_id = '20230518T155901_20230518T160728_T17RLL'
glinted_image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/'+image_id)

In [ ]:
## This is a predefined polygon with glinted pixels as example
## The user can use geemap to create a different polygon
sunglint = (ee.Geometry.Polygon(
        [[[-82.5882342558702, 27.68003130872319],
          [-82.5882342558702, 27.677903068330107],
          [-82.58548767383895, 27.677903068330107],
          [-82.58548767383895, 27.68003130872319]]], None, False))

## Visualize image
vis_params = {
    'min': 0,
    'max': 2000,
    'bands': ['B4', 'B3', 'B2']
}
Map = geemap.Map()
Map.centerObject(sunglint, 13)
Map.addLayer(glinted_image, vis_params, 'Glinted Image')
Map.addLayer(sunglint, {'color':'red'}, 'Glinted Area')
Map

### **Preprocessing and deglinting**

In [ ]:
## Rescale reflectance
img_rescale = functions.rescale(glinted_image)

## Apply cloud mask -only if necessary- using reduced sensitivity (e.g. 20)
img_cloud = functions.CloudScore6S(ee.ImageCollection.fromImages([img_rescale]), 20)

## Apply deglint function
deglinted = functions.deglint(img_cloud.first(), sunglint)

## Visualize image
vis_params_2 = {
    'min': 0,
    'max': 0.2,
    'bands': ['B4', 'B3', 'B2']
}
Map = geemap.Map()
Map.centerObject(deglinted, 11)
Map.addLayer(deglinted, vis_params_2, 'Deglinted Image')
Map

The deglinting process might be "aggressive" in some cases, producing noise and negative reflectance in B4

In [ ]:
## Clean negative values using B4.
deglintMask = deglinted.select('B4').gte(0)
deglintCleaned = deglinted.updateMask(deglintMask)

### **Export image to Assets:**

In [17]:
task = ee.batch.Export.image.toAsset(
  image= deglintCleaned,
  description= 'Deglinted_'+id,
  assetId= asset_id + id,
  region= glinted_image.geometry(),
  crs= 'EPSG:4326',
  scale= 10,
  maxPixels= 1e13
)
task.start()